# 4. tutorial - High Dynamic Range (HDR)
[High Dynamic Range Imaging](https://en.wikipedia.org/wiki/High-dynamic-range_imaging) refers to the set of imaging technologies and techniques that allow to increase the dynamic range of images or videos.

In [ ]:
import exif
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import requests
import shutil
import os
import tempfile

### Helper functions for images loading

In [ ]:
def download_images(urls, path):
    for idx in range(len(urls)):
        print(urls[idx], end='')
        r = requests.get(urls[idx], stream=True)
        if r.status_code == 200:
            with open(os.path.join(path, f'image{idx:02d}.jpg'), 'bw') as f:
                # r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)    
                print('...OK')
        else:
                print('...FAIL')

def read_images(file_pattern, scale_percent = 100):
    files = glob.glob(file_pattern) # Načtení cest ke všem souborům dle daného vzoru

    imgs = []
    t = []

    # cyklus přes všechny soubory (obrázky v dané cestě)
    for file in files:
        tmp_img = cv2.imread(file) # Načtení obrázku
        width = int(tmp_img.shape[1] * scale_percent // 100) #nový rozměr po zmenšení dle dané hodnoty v %
        height = int(tmp_img.shape[0] * scale_percent // 100) #nový rozměr po zmenšení dle dané hodnoty v %
        dim = (width, height) #nový rozměr po zmenšení dle dané hodnoty v %
        imgs.append(cv2.resize(tmp_img, dim, interpolation = cv2.INTER_AREA)) # zmenšení obrázku a uložení do listu
        info = exif.Image(file) # Načtení EXIF z obrázku
        t.append(info.exposure_time) # Uložení času expozice do listu
    return imgs, t # Návrat velikosti obrázků, pole s obrázky a vektoru časů expozic

### Load images from github

In [ ]:
images_urls = ['https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka02.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka03.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka04.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka05.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka06.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka07.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka08.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka09.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka10.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka11.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka12.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka13.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka14.jpg?raw=true',
               'https://github.com/CVUT-FS-12110/Machine-Perception-and-Image-Analysis/blob/master/src/lectures/04_HDR/images/lampicka15.jpg?raw=true',
              ]

with tempfile.TemporaryDirectory() as tempdir:
    download_images(images_urls, tempdir)
    images, times = read_images(os.path.join(tempdir, "*.jpg"), scale_percent = 100)
    
print(f'Exposure times [s]: {t}')

### Show all images and exposure times

In [ ]:
for idx in range(len(t)):
    plt.figure()
    plt.imshow(cv2.cvtColor(images[idx], cv2.COLOR_BGR2RGB))
    plt.title('{} s'.format(t[idx]))

### Create HDR image by Debevec method

In [ ]:
times = np.array(t[-1::-1], dtype=np.float32)


calibrate = cv2.createCalibrateDebevec()
response = calibrate.process(images[-1::-1], times=times)

merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(images[-1::-1], times=times, response=response)

plt.plot(response.squeeze()[:,0], 'b')
plt.plot(response.squeeze()[:,1], 'g')
plt.plot(response.squeeze()[:,2], 'r')
plt.title('Camera response function');

In [ ]:
plt.imshow(hdr_debevec)
plt.title('HDR Image - clipped');


### HDR to LDR by tonemapping

In [ ]:
tonemap = cv2.createTonemap(gamma=8)
ldr_debevec = tonemap.process(hdr_debevec)
plt.imshow(cv2.cvtColor(ldr_debevec, cv2.COLOR_BGR2RGB))

### HDR alternative, Exposure Fusion
Exposure Fusion blend images with different exposure times (without knowledge of them). Looks better on display, but never produce HDR, it is produce LDR directly.

In [ ]:
merge_martens = cv2.createMergeMertens()
ldr_martens = merge_martens.process(images)

In [ ]:
plt.imshow(cv2.cvtColor(ldr_martens, cv2.COLOR_BGR2RGB))